# Multi-Model Analysis : Priestley-Taylor (PETPT) and Penaman-Monteith Based Models (PETPNO, PETPEN, PETDYN)
This notebook focuses on answering the question: how can AutoMATES leverage the information extracted from source code and free text to improve modelers ability to visualize input sensitivity in a multiple crop models? We shall demonstrate the ability of AutoMATES to analyze these models from the DSST codebase. In addition, results of a comparative study of various models using global sensitivity analysis is also included in the document.

## Visualization goals
- Show that S1 analysis over the whole range of all variables demonstrates a high S1 value for TMAX
- Show that we can use static program analysis to determine that TMAX controls a piecewise function
- Show that we can use evidence from TR when breaking up the bound range of TMAX to reflect the piecewise nature of the function
- Show the updated S1/S2 plots for the broken down ranges

In [ ]:
from penman import SensitivityModel
from model_analysis.visualization import SensitivityVisualizer
from IPython.display import display

##### List of Sample Sizes #####

sample_list = [10**x for x in range(1, 6)]


# Setting SA method to deploy
method = 'Sobol'


In [ ]:
# Choice of Model
model = 'PETPT'

# Using the full-range bounds at first
bounds = {
        'tmax':[-10.0, 50.0],
        'tmin':[0.0, 23.9],
        'srad': [2.45, 27.8],
        'msalb': [0.18, 0.2],
        'xhlai': [0.0, 4.77]
    }

SM = SensitivityModel(model, bounds, sample_list, method)

decimal=4
df_PETPT_S1, df_PETPT_ST = SM.generate_dataframe(decimal)

indices_lst = SM.generate_indices()

component = 'S1'
SM.sensitivity_plots(indices_lst, component)
component = 'S2'
SM.sensitivity_plots(indices_lst, component)

### Analysis of the above results
As we can see from the S1 plot abovwe, it appears that TMAX and SRAD play critical roles in determining the uncertainty of the output here. We can inspect the source code using AutoMATES to try and determine why these two variables play such a significant role. Below is a code-block from the PETPT Fortran source code:
```fortran
      SLANG = SRAD*23.923
      EEQ = SLANG*(2.04E-4-1.83E-4*ALBEDO)*(TD+29.0)
      EO = EEQ*1.1

      IF (TMAX .GT. 35.0) THEN
        EO = EEQ*((TMAX-35.0)*0.05+1.1)
      ELSE IF (TMAX .LT. 5.0) THEN
        EO = EEQ*0.01*EXP(0.18*(TMAX+20.0))
      ENDIF

      EO = MAX(EO,0.0001)
```
In this code-block we can see that TMAX plays a critical role at the end of the PETPT function. This role is to divide the function into three piecewise components based upon the value of TMAX. AutoMATES can detect this using static code analysis on the conditional variables included in the model code.

Using this along with text-reading knowledge of the parameter bounds for TMAX, we can divide the total range of values for TMAX into three separate bound ranges that we can use for further analysis:
1. -10.0ºC -- 5.0ºC
2. 5.0ºC -- 35.0ºC
3. 35.0ºC -- 55.0ºC

Below we show the S1/S2 results for these three different bound ranges.

### Result Set 1 - TMAX : [5.0,  35.0]
Notice for this first result set that we are working with non-extreme values of TMAX. When the values for TMAX are no longer extreme, we see that it drops off significantly from the S1 indices and allows other variables to show how much more of an important role they play in determining model output.

#### Key insights
- SRAD now dominates all other uncertainty sources in the S1 indices
- The interaction between SRAD and TMAX is now shown to be an important S2 index

In [ ]:
# Choosing the bound set
bounds = {
        'tmax':[5.0, 35.0],
        'tmin':[0.0, 23.9],
        'srad': [2.45, 27.8],
        'msalb': [0.18, 0.2],
        'xhlai': [0.0, 4.77]
    }

SM = SensitivityModel(model, bounds, sample_list, method)

decimal = 4
df_PETPT_S1_set1, df_PETPT_ST_set1 = SM.generate_dataframe(decimal)

indices_lst = SM.generate_indices()

SM.sensitivity_plots(indices_lst, "S1")
SM.sensitivity_plots(indices_lst, "S2")

### Result Set 2 - TMAX : [-10.0,  4.99]
For this result set we are looking at the extreme lower end of values for TMAX. Here we see that TMAX is an even greater contributor to S1 sensitivity than over the whole range. This is likely due to the scaling of EO done by an exponential term involving TMAX in this portion of the piecewise function.

In [ ]:
bounds = {
        'tmax':[-10.0, 4.99],
        'tmin':[0.0, 23.9],
        'srad': [2.45, 27.8],
        'msalb': [0.18, 0.2],
        'xhlai': [0.0, 4.77]
    }

SM = SensitivityModel(model, bounds, sample_list, method)

decimal=4
df_PETPT_S1_set2, df_PETPT_ST_set2 = SM.generate_dataframe(decimal)

indices_lst = SM.generate_indices()

SM.sensitivity_plots(indices_lst, "S1")
SM.sensitivity_plots(indices_lst, "S2")

### Result Set 3 - TMAX : [35.1,  55.0]
Here we are testing the extreme high end of values for TMAX. For this bound range we are now observing:
- SRAD is the largest contributor to the S1 sensitivity
- TMAX is still a large contributor to the S1 sensitivity
- The total S1 sensitivity has increased dramatically, causing a large decrease in S2 total sensitivity which lowers the impact of the TMAX/SRAD interaction term. This is likely due to the scaling of EO by TMAX in this branch of the piecewise function

In [ ]:
bounds = {
        'tmax':[35.1, 55.0],
        'tmin':[0.0, 23.9],
        'srad': [2.45, 27.8],
        'msalb': [0.18, 0.2],
        'xhlai': [0.0, 4.77]
    }

SM = SensitivityModel(model, bounds, sample_list, method)

decimal=4
df_PETPT_S1_set3, df_PETPT_ST_set3 = SM.generate_dataframe(decimal)

indices_lst = SM.generate_indices()

component = 'S1'
SM.sensitivity_plots(indices_lst, component)
component = 'S2'
SM.sensitivity_plots(indices_lst, component)

### XHLAI : [-5.0, 5.0]; TMAX : [5.0, 35.0]

In [ ]:
# Choosing the bound set
bounds = {
        'tmax':[5.0, 35.0],
        'tmin':[0.0, 23.9],
        'srad': [2.45, 27.8],
        'msalb': [0.18, 0.2],
        'xhlai': [-5.0, 5.0]
    }

SM = SensitivityModel(model, bounds, sample_list, method)
decimal=4
df_PETPT_S1_xhlai, df_PETPT_ST_xhlai = SM.generate_dataframe(decimal)
indices_lst = SM.generate_indices()

SM.sensitivity_plots(indices_lst, "S1")
SM.sensitivity_plots(indices_lst, "S2")

### SRAD : [0.0, 5.0]; TMAX : [5.0, 35.0]

In [ ]:
model='PETPT'
bounds = {
        'tmax':[5.0, 35.0],
        'tmin':[0.0, 23.9],
        'srad': [0.0, 5.0],
        'msalb': [0.18, 0.2],
        'xhlai': [0.0, 4.77]
    }

SM = SensitivityModel(model, bounds, sample_list, method)

decimal=4
df_PETPT_S1_srad, df_PETPT_ST_srad = SM.generate_dataframe(decimal)

indices_lst = SM.generate_indices()

SM.sensitivity_plots(indices_lst, "S1")
SM.sensitivity_plots(indices_lst, "S2")

## PETPNO

### Result Set 1 - TMAX  : [5.0,  35.0]

- SRAD still has the highest S1 index
- TMAX, TMIN, TDEW, CLOUDS all have moderately strong indices
- (TMAX, SRAD), (TDEW, SRAD) have S2 indices above 0.01 cut-off

In [ ]:
model = 'PETPNO'

bounds = {
        'tmax':[5.0, 35.0],
        'tmin':[0.0, 23.9],
        'srad': [2.45, 27.8],
        'msalb': [0.18, 0.2],
        'xhlai': [0.0, 4.77],
        'tavg': [8.0, 31],
        'tdew': [8.0, 31],
        'windsp': [0.0, 10.0],
        'clouds': [0.0, 1.0]
    }

SM = SensitivityModel(model, bounds, sample_list, method)

decimal=4
df_PETPNO_S1_set1, df_PETPNO_ST_set1 = SM.generate_dataframe(decimal)

indices_lst = SM.generate_indices()

SM.sensitivity_plots(indices_lst, "S1")
SM.sensitivity_plots(indices_lst, "S2")

### Result Set 2  -  TMAX : [-10.0,  4.99]

- SRAD still has the largest S1 index but lower than in the previous case
- TMAX, TMIN have large S1 indices
- (TMIN, SRAD), (TDEW, SRAD) S2 indices have increased significantly

In [ ]:
bounds = {
        'tmax':[-10.0, 4.99],
        'tmin':[0.0, 23.9],
        'srad': [2.45, 27.8],
        'msalb': [0.18, 0.2],
        'xhlai': [0.0, 4.77],
        'tavg': [8.0, 31],
        'tdew': [8.0, 31],
        'windsp': [0.0, 10.0],
        'clouds': [0.0, 1.0]
    }

SM = SensitivityModel(model, bounds, sample_list, method)

decimal=4
df_PETPNO_S1_set2, df_PETPNO_ST_set2 = SM.generate_dataframe(decimal=4)

indices_lst = SM.generate_indices()

SM.sensitivity_plots(indices_lst, "S1")
SM.sensitivity_plots(indices_lst, "S2")

### Result Set 3 - TMAX : [35.1,  55.0]

- SRAD has maximum S1 and close to 1.0 signifying that the majority of contribution to sensitivity stems from this parameter 

- CLOUDS, TMAX has moderately strong S1 index

In [ ]:
bounds = {
        'tmax':[35.1, 55.0],
        'tmin':[0.0, 23.9],
        'srad': [2.45, 27.8],
        'msalb': [0.18, 0.2],
        'xhlai': [0.0, 4.77],
        'tavg': [8.0, 31],
        'tdew': [8.0, 31],
        'windsp': [0.0, 10.0],
        'clouds': [0.0, 1.0]
    }

SM = SensitivityModel(model, bounds, sample_list, method)

decimal=4
df_PETPNO_S1_set3, df_PETPNO_ST_set3 = SM.generate_dataframe(decimal)

indices_lst = SM.generate_indices()

SM.sensitivity_plots(indices_lst, "S1")
SM.sensitivity_plots(indices_lst, "S2")

### XHLAI : [-5.0,  5.0] ; TMAX : [5.0,  35.0]

There exists a conditional (shown below) in PETPNO which alters the function of EO. We take negative values of XHLAI to probe whether it affects the analysis. TMAX values have been taken from Result Set 1

- No significant change is observed

``` fortran
      IF (XHLAI .LE. 0.0) THEN
        ALBEDO = MSALB
      ELSE
        ALBEDO = 0.23-(0.23-MSALB)*EXP(-0.75*XHLAI)
      ENDIF

```

In [ ]:
bounds = {
        'tmax':[5.0, 35.0],
        'tmin':[0.0, 23.9],
        'srad': [2.45, 27.8],
        'msalb': [0.18, 0.2],
        'xhlai': [-5.0, 5.0],
        'tavg': [8.0, 31],
        'tdew': [8.0, 31],
        'windsp': [0.0, 10.0],
        'clouds': [0.0, 1.0]
    }

SM = SensitivityModel(model, bounds, sample_list, method)

decimal=4
df_PETPNO_S1_xhlai, df_PETPNO_ST_xhlai = SM.generate_dataframe(decimal)

indices_lst = SM.generate_indices()

SM.sensitivity_plots(indices_lst, "S1")
SM.sensitivity_plots(indices_lst, "S2")

### SRAD : [0.0,  5.0];  TMAX : [5.0,  35.0]

Severely restricting the range of SRAD changes the S1 vs log(N) profile appreciably. This is interesting since the only place SRAD appears in the code is in the assignment of RNET :
```fortran
      RNET= (1.0-ALBEDO)*SRAD - RADB
```

What it simply tells us is that the extent to which SRAD can contribute to the variance in the output depends enormously on its bounds.

In [ ]:
model = 'PETPNO'
bounds = {
        'tmax':[5.0, 35.0],
        'tmin':[0.0, 23.9],
        'srad': [0.0, 5.0],
        'msalb': [0.18, 0.2],
        'xhlai': [0.0, 4.77],
        'tavg': [8.0, 31],
        'tdew': [8.0, 31],
        'windsp': [0.0, 10.0],
        'clouds': [0.0, 1.0]
    }

SM = SensitivityModel(model, bounds, sample_list, method)

decimal=4
df_PETPNO_S1_srad, df_PETPNO_ST_srad = SM.generate_dataframe(decimal)

indices_lst = SM.generate_indices()

SM.sensitivity_plots(indices_lst, "S1")
SM.sensitivity_plots(indices_lst, "S2")

## PETPEN 

### Result Set 1  -  TMAX : [5.0,  35.0}

- SRAD yet again has the highest S1 index
- EORATIO has a large S1 index unqiue to PETPEN
- (VAPR, CLOUDS), (EORATIO, XHLAI), (EORATIO, SRAD) have significant S2 indices 

In [ ]:
model = 'PETPEN'

bounds = {
        'tmax':[5.0, 35.0],
        'tmin':[0.0, 23.9],
        'srad': [2.45, 27.8],
        'msalb': [0.18, 0.2],
        'xhlai': [0.0, 4.77],
        'tavg': [8.0, 31],
        'tdew': [8.0, 31],
        'windsp': [0.0, 10.0],
        'windht': [1.0, 25.0],
        'vapr': [0.0, 20.0],
        'clouds': [0.0, 1.0],
        'eoratio': [0.0, 2.0]
}



SM = SensitivityModel(model, bounds, sample_list, method)

decimal=4
df_PETPEN_S1_set1, df_PETPEN_ST_set1 = SM.generate_dataframe(decimal)

indices_lst = SM.generate_indices()

SM.sensitivity_plots(indices_lst, "S1")
SM.sensitivity_plots(indices_lst, "S2")

### Result Set 2  -  TMAX : [-10.0,  4.99]

- SRAD still has the highest S1 index
- S2 values of relevant pairs are similar

In [ ]:
bounds = {
        'tmax':[-10.0, 4.99],
        'tmin':[0.0, 23.9],
        'srad': [2.45, 27.8],
        'msalb': [0.18, 0.2],
        'xhlai': [0.0, 4.77],
        'tavg': [8.0, 31],
        'tdew': [8.0, 31],
        'windsp': [0.0, 10.0],
        'windht': [1.0, 25.0],
        'vapr': [0.0, 20.0],
        'clouds': [0.0, 1.0],
        'eoratio': [0.0, 2.0]
}



SM = SensitivityModel(model, bounds, sample_list, method)

decimal=4
df_PETPEN_S1_set2, df_PETPEN_ST_set2 = SM.generate_dataframe(decimal)

indices_lst = SM.generate_indices()

SM.sensitivity_plots(indices_lst, "S1")
SM.sensitivity_plots(indices_lst, "S2")

### Result Set 3  -  TMAX : [35.1,  55.0]

-  SRAD has the highest S1
- Trend in S2 does not appear to change appreciably

In [ ]:
bounds = {
        'tmax':[35.1, 55.0],
        'tmin':[0.0, 23.9],
        'srad': [2.45, 27.8],
        'msalb': [0.18, 0.2],
        'xhlai': [0.0, 4.77],
        'tavg': [8.0, 31],
        'tdew': [8.0, 31],
        'windsp': [0.0, 10.0],
        'windht': [1.0, 25.0],
        'vapr': [0.0, 20.0],
        'clouds': [0.0, 1.0],
        'eoratio': [0.0, 2.0]
}



SM = SensitivityModel(model, bounds, sample_list, method)

decimal=4
df_PETPEN_S1_set3, df_PETPEN_ST_set3 = SM.generate_dataframe(decimal)

indices_lst = SM.generate_indices()

SM.sensitivity_plots(indices_lst, "S1")
SM.sensitivity_plots(indices_lst, "S2")

### XHLAI : [-5.0,  5.0] ; TMAX : [5.0,  35.0]

We repeat the same trials as was done in previous models where based on conditional statements, we vary the input parameter interval ranges to see if that causes any change in the ranking of the sensitivity indices.

```fortran
      IF (XHLAI .LE. 0.0) THEN
        ALBEDO = MSALB
      ELSE
        ALBEDO = 0.23-(0.23-SALB)*EXP(-0.75*XHLAI)
        ALBEDO = 0.23
      ENDIF
```

In [ ]:
bounds = {
        'tmax':[5.0, 35.0],
        'tmin':[0.0, 23.9],
        'srad': [2.45, 27.8],
        'msalb': [0.18, 0.2],
        'xhlai': [-5.0, 5.0],
        'tavg': [8.0, 31],
        'tdew': [8.0, 31],
        'windsp': [0.0, 10.0],
        'windht': [1.0, 25.0],
        'vapr': [0.0, 20.0],
        'clouds': [0.0, 1.0],
        'eoratio': [0.0, 2.0]
}



SM = SensitivityModel(model, bounds, sample_list, method)

decimal=4
df_PETPEN_S1_xhlai, df_PETPEN_ST_xhlai = SM.generate_dataframe(decimal)

indices_lst = SM.generate_indices()

SM.sensitivity_plots(indices_lst, "S1")
SM.sensitivity_plots(indices_lst, "S2")

### SRAD:  [0.0,  5.0] ; TMAX : [5.0,  35.0]

-  Changing the interval range of SRAD dramatically alters the Sobol index ranking. S1 contribution from SRAD is decreased significantly.

-  Again, if one looks at the PETPEN Fortran code, SRAD appears in the assignment of RNET as seen in PETPNO

```fortran
      RNET= (1.0-ALBEDO)*SRAD - RADB
```

In [ ]:
bounds = {
        'tmax':[5.0, 35.0],
        'tmin':[0.0, 23.9],
        'srad': [0.0, 5.0],
        'msalb': [0.18, 0.2],
        'xhlai': [0.0, 4.77],
        'tavg': [8.0, 31],
        'tdew': [8.0, 31],
        'windsp': [0.0, 10.0],
        'windht': [1.0, 25.0],
        'vapr': [0.0, 20.0],
        'clouds': [0.0, 1.0],
        'eoratio': [0.0, 2.0]
}



SM = SensitivityModel(model, bounds, sample_list, method)

decimal=4
df_PETPEN_S1_srad, df_PETPEN_ST_srad = SM.generate_dataframe(decimal)

indices_lst = SM.generate_indices()

SM.sensitivity_plots(indices_lst, "S1")
SM.sensitivity_plots(indices_lst, "S2")

## PETDYN

### Result Set 1 - TMAX : [5.0,  35.0]

-  SRAD has an overwhelming contribution to the overall variance
- All other sobol indices are relatively insignificant in comparison to SRAD 

In [ ]:
model = 'PETDYN'

bounds = {
        'tmax':[5.0, 35.0],
        'tmin':[0.0, 23.9],
        'srad': [2.45, 27.8],
        'msalb': [0.18, 0.2],
        'xhlai': [0.0, 4.77],
        'tavg': [8.0, 31],
        'tdew': [8.0, 31],
        'windsp': [0.0, 10.0],
        'clouds': [0.0, 1.0],
        'canht': [0.0, 5.0]
}

SM = SensitivityModel(model, bounds, sample_list, method)

decimal=4
df_PETDYN_S1_set1, df_PETDYN_ST_set1 = SM.generate_dataframe(decimal)

indices_lst = SM.generate_indices()

SM.sensitivity_plots(indices_lst, "S1")
SM.sensitivity_plots(indices_lst, "S2")

### Result Set 2 - TMAX : [-10.0,  4.99]

- Even with a different range of TMAX, S1 from SRAD is the largest. 

In [ ]:
bounds = {
        'tmax':[-10.0, 4.99],
        'tmin':[0.0, 23.9],
        'srad': [2.45, 27.8],
        'msalb': [0.18, 0.2],
        'xhlai': [0.0, 4.77],
        'tavg': [8.0, 31],
        'tdew': [8.0, 31],
        'windsp': [0.0, 10.0],
        'clouds': [0.0, 1.0],
        'canht': [0.0, 5.0]
}

SM = SensitivityModel(model, bounds, sample_list, method)

decimal=4
df_PETDYN_S1_set2, df_PETDYN_ST_set2 = SM.generate_dataframe(decimal)

indices_lst = SM.generate_indices()

SM.sensitivity_plots(indices_lst, "S1")
SM.sensitivity_plots(indices_lst, "S2")

### Result Set 3 - TMAX : [35.1,  55.0]

- Across all ranges of TMAX, SRAD has the highest S1

In [ ]:
bounds = {
        'tmax':[35.1, 55.0],
        'tmin':[0.0, 23.9],
        'srad': [2.45, 27.8],
        'msalb': [0.18, 0.2],
        'xhlai': [0.0, 4.77],
        'tavg': [8.0, 31],
        'tdew': [8.0, 31],
        'windsp': [0.0, 10.0],
        'clouds': [0.0, 1.0],
        'canht': [0.0, 5.0]
}

SM = SensitivityModel(model, bounds, sample_list, method)

decimal=4
df_PETDYN_S1_set3, df_PETDYN_ST_set3 = SM.generate_dataframe(decimal)

indices_lst = SM.generate_indices()

SM.sensitivity_plots(indices_lst, "S1")
SM.sensitivity_plots(indices_lst, "S2")

### SRAD : [0.0,  5.0] ; TMAX : [5.0,  35.0]

- Severely constraining/restricting the interval range of SRAD, changes the S1 values of the parameters.
- Most notably, SRAD S1 is lowered well below its previous values. S1 due to CLOUDS begin to dominate with the largest S1 index.
- In the S2 matrix, (CLOUDS, SRAD) and (CLOUDS, TDEW) yield moderately large S2 values.

In [ ]:
bounds = {
        'tmax':[5.0, 35.0],
        'tmin':[0.0, 23.9],
        'srad': [0.0, 5.0],
        'msalb': [0.18, 0.2],
        'xhlai': [0.0, 4.77],
        'tavg': [8.0, 31],
        'tdew': [8.0, 31],
        'windsp': [0.0, 10.0],
        'clouds': [0.0, 1.0],
        'canht': [0.0, 5.0]
}

SM = SensitivityModel(model, bounds, sample_list, method)

decimal=4
df_PETDYN_S1_srad, df_PETDYN_ST_srad = SM.generate_dataframe(decimal)

indices_lst = SM.generate_indices()

SM.sensitivity_plots(indices_lst, "S1")
SM.sensitivity_plots(indices_lst, "S2")

## PETASCE

### Result Set 1 - TMAX : [5.0,  35.0]

-  SRAD has the largest S1 index
-  In a slight departure from what has been observed up until now, XHLAI has a fairly contribution to the response variance.
- The interaction of SRAD and XHLAI yields an S2 index above the threshol of. 0.01.

In [ ]:
model = 'PETASCE'

bounds = {
        'tmax':[5.0, 35.0],
        'tmin':[0.0, 23.9],
        'srad': [2.45, 27.8],
        'msalb': [0.18, 0.2],
        'xhlai': [0.0, 4.77],
        'tdew': [8.0, 31],
        'windht': [1.0, 25.0],
        'windrun': [0, 900],
        'xlat': [-65.0, 65.0],
        'xelev': [0, 6000],
        'canht': [0.0, 5.0],
        'doy': [1, 365],
        'meevp': [0, 1],
}


SM = SensitivityModel(model, bounds, sample_list, method)

decimal=4
df_PETASCE_S1_set1, df_PETASCE_ST_set1 = SM.generate_dataframe(decimal)

indices_lst = SM.generate_indices()

SM.sensitivity_plots(indices_lst, "S1")
SM.sensitivity_plots(indices_lst, "S2")

### Result Set 2 - TMAX : [-10.0,  4.99]

- SRAD followed by XHLAI still have the highest S1 indices while (SRAD, XHLAI) pair has the largest S2 index.

In [ ]:
bounds = {
        'tmax':[-10.0, 4.99],
        'tmin':[0.0, 23.9],
        'srad': [2.45, 27.8],
        'msalb': [0.18, 0.2],
        'xhlai': [0.0, 4.77],
        'tdew': [8.0, 31],
        'windht': [1.0, 25.0],
        'windrun': [0, 900],
        'xlat': [-65.0, 65.0],
        'xelev': [0, 6000],
        'canht': [0.0, 5.0],
        'doy': [1, 365],
        'meevp': [0, 1],
}


SM = SensitivityModel(model, bounds, sample_list, method)

decimal=4
df_PETASCE_S1_set2, df_PETASCE_ST_set2 = SM.generate_dataframe(decimal)

indices_lst = SM.generate_indices()

SM.sensitivity_plots(indices_lst, "S1")
SM.sensitivity_plots(indices_lst, "S2")

### Result Set 3 -  TMAX : [35.1,  55.0]

In [ ]:
bounds = {
        'tmax':[35.1, 55.0],
        'tmin':[0.0, 23.9],
        'srad': [2.45, 27.8],
        'msalb': [0.18, 0.2],
        'xhlai': [0.0, 4.77],
        'tdew': [8.0, 31],
        'windht': [1.0, 25.0],
        'windrun': [0, 900],
        'xlat': [-65.0, 65.0],
        'xelev': [0, 6000],
        'canht': [0.0, 5.0],
        'doy': [1, 365],
        'meevp': [0, 1],
}


SM = SensitivityModel(model, bounds, sample_list, method)

decimal=4
df_PETASCE_S1_set3, df_PETASCE_ST_set3 = SM.generate_dataframe(decimal)

indices_lst = SM.generate_indices()

SM.sensitivity_plots(indices_lst, "S1")
SM.sensitivity_plots(indices_lst, "S2")

### XHLAI : [-5.0,  5.0]; TMAX : [5.0,   35.0]

Varying the lower bound of XHLAI produces interesting results. We should expect slight changes in the values of indices since it has already been established that XHLAI is an important factor while determining output variance. Static code analysis reveals that condiitonal statements within the code involves XHLAI in two different places as shown below :

```fortran
      IF (XHLAI .LE. 0.0) THEN
        ALBEDO = MSALB
      ELSE
        ALBEDO = 0.23
      ENDIF
      RNS = (1.0-ALBEDO)*SRAD !MJ/m2/d
```

```fortran
      IF (XHLAI .LE. 0.0) THEN
         KCB = 0.0
      ELSE
         !DeJonge et al. (2012) equation
         KCB = MAX(0.0,KCBMIN+(KCBMAX-KCBMIN)*(1.0-EXP(-1.0*SKC*XHLAI)))
      ENDIF
```

These dependencies affect the outcome while computing Sobol indices.

- XHLAI index is raised significantly while S1 of SRAD is lowered simultaneously in this regime. However, SRAD's index is still higher than that of XHLAI as was seen in Penman-Monteith models.

In [ ]:
bounds = {
        'tmax':[5.0, 35.0],
        'tmin':[0.0, 23.9],
        'srad': [2.45, 27.8],
        'msalb': [0.18, 0.2],
        'xhlai': [-5.0, 5.0],
        'tdew': [8.0, 31],
        'windht': [1.0, 25.0],
        'windrun': [0, 900],
        'xlat': [-65.0, 65.0],
        'xelev': [0, 6000],
        'canht': [0.0, 5.0],
        'doy': [1, 365],
        'meevp': [0, 1],
}


SM = SensitivityModel(model, bounds, sample_list, method)

decimal=4
df_PETASCE_S1_xhlai, df_PETASCE_ST_xhlai = SM.generate_dataframe(decimal)

indices_lst = SM.generate_indices()

SM.sensitivity_plots(indices_lst, "S1")
SM.sensitivity_plots(indices_lst, "S2")

### SRAD : [0.0,  5.0] ; TMAX : [5.0,  35.0]

In [ ]:
bounds = {
        'tmax':[5.0, 35.0],
        'tmin':[0.0, 23.9],
        'srad': [0.0, 5.0],
        'msalb': [0.18, 0.2],
        'xhlai': [0.0, 4.77],
        'tdew': [8.0, 31],
        'windht': [1.0, 25.0],
        'windrun': [0, 900],
        'xlat': [-65.0, 65.0],
        'xelev': [0, 6000],
        'canht': [0.0, 5.0],
        'doy': [1, 365],
        'meevp': [0, 1],
}


SM = SensitivityModel(model, bounds, sample_list, method)

decimal=4
df_PETASCE_S1_srad, df_PETASCE_ST_srad = SM.generate_dataframe(decimal)

indices_lst = SM.generate_indices()

SM.sensitivity_plots(indices_lst, "S1")
SM.sensitivity_plots(indices_lst, "S2")

## Take Home Message

In this demo we have shown how an initial sensitivity analysis of a scientific model can inform which variables to search for when doing single-variable range analysis. AutoMATES enables us to search the source code of the model automatically to determine where in the code variables with high S1 sensitivites cause functional differences. Using the parameter estimation from the TR component of AutoMATES and static progam analysis from the PA component of AutoMATES we can then perform a second set of sensitivty analysis upon a broken up range of the variable in question. This allows AutoMATES to deliver deeper insights about the sensitivity of a scientific model to its inputs without requiring a modeler to:
- manualy trek through the model code searching for the variable interactions of highest interest
- find correct bound parameters for variables of interest in the literature
- create new sensitivity analysis experiments as appropriate for the new ranges of a variable of interest

Key Insights:
- TMAX is the most important factor in Priestley-Taylor model
- Changes in the interval range of TMAX can significantly modify the order of Sobol indices.  For example, in the interval range [-10.0, 4.99], SRAD has a higher Sobol index.
- SRAD is the most dominant factor in all Penman-Monteith based models as well as in PETASCE. 
- Changes in TMAX or XHLAI do not alter the Ranking of Sensitivity Indices of SRAD  in PETPNO, PETPEN, PETDYN, and. PETASCE.
- In Penman models, changes in the interval ranges of SRAD can significantly alter the first and second order indices of SRAD as expected. Static code analysis however shows that SRAD do not appear in any conditional statements that will lead to evaulation of the response funciton (EO) in a piecewise manner. In all three cases, SRAD appears in the assignment of RNET and EO shares a linear relationship with RNET and hence SRAD.

PETPNO
--
``` fortran
      RNET= (1.0-ALBEDO)*SRAD - RADB
      WFNFAO = 0.0027 * (1.0+0.01*WINDSP)
      RNETMG = (RNET-G) / LHVAP * 1.0E6
      EO = (S*RNETMG + PSYCON*WFNFAO*VPD) / (S+PSYCON)
```
PETPEN
--

```fortran
      RNET= (1.0-ALBEDO)*SRAD - RADB
      RNETMG = (RNET-G)
      ET0 = ((S*RNETMG + (DAIR*SHAIR*VPD)/ra)/(S+PSYCON*(1+rs/ra)))
      ET0 = ET0/ (LHVAP / 1000000.)
      KC=1.0+(EORATIO-1.0)*MIN(XHLAI, 6)/6.0
      EO=ET0*KC
```
PETDYN
--
```fortran
      RNET= (1.0-ALBEDO)*SRAD - RADB
      RNETMG = (RNET-G)      
      EO=((S*RNETMG + (DAIR*SHAIR*VPD)/RAERO)/(S+PSYCON*(1+RTOT/RAERO))) (PETDYN)
```

- In PETASCE, changes in SRAD/TMAX/XHLAI interval bounds do not alter the ranking of indices at all.

## Results for TMAX : [5.0, 35.0]

- Tables below with a list of ST (total sensitivity index due to a parameter) and the first order indices (S1) for each model - PETPT, PETPNO, PETPEN, PETDYN, and PETASCE have been provided can summarize our results.
- SRAD in all cases has the highest ST and S1 indices within this range of TMAX

## 1. ST

In [ ]:
from soboltable import table

frames = [df_PETPT_ST_set1, df_PETPNO_ST_set1, df_PETPEN_ST_set1, df_PETDYN_ST_set1, df_PETASCE_ST_set1]

Table = table(frames)
Table.style.highlight_max(color='lightgreen', axis=1)


## 2. S1

In [ ]:
frames = [df_PETPT_S1_set1, df_PETPNO_S1_set1, df_PETPEN_S1_set1, df_PETDYN_S1_set1, df_PETASCE_S1_set1]
Table = table(frames)
Table.style.highlight_max(color='lightgreen', axis=1)

## Results for TMAX : [-10.0,  4.99]

- Using a different interval range of TMAX, PETPT S1 and ST values change considerably. Unlike the previous table, we see that TMAX in PETPT has the highest ST and S1. 
- However, in Peman-Monteith based models, i.e., PETPNO, PETPEN, PETDYN, SRAD still has the highest ST and S1 in all cases. 
- This is because that while TMAX and SRAD are the most influential parameters in PETPT and Penman models (PETPNO, PETPEN, PETDYN). As a result, changes in interval ranges of TMAX do not affect the ordering of Sobol indices in the latter models whereas it has a massive effect on the indices of PETPT parameters.

## ST

In [ ]:
frames = [df_PETPT_ST_set2, df_PETPNO_ST_set2, df_PETPEN_ST_set2, df_PETDYN_ST_set2, df_PETASCE_ST_set2]

Table = table(frames)
Table.style.highlight_max(color='lightgreen', axis=1)

## S1

In [ ]:
frames = [df_PETPT_S1_set2, df_PETPNO_S1_set2, df_PETPEN_S1_set2, df_PETDYN_S1_set2, df_PETASCE_S1_set2]
Table = table(frames)
Table.style.highlight_max(color='lightgreen', axis=1)

## Results for SRAD : [0.0,  5.0] ; TMAX : [5.0,  35.0]

- Altering the interval ranges of SRAD has a huge impact on PETPNO, PETPEN, and PETDYN. Table below shows that CLOUDS has the highest Sobol indices (both ST and S1).
- Modifying the range of SRAD does not affect the ranking in PETPT or PETASCE

## ST

In [ ]:
frames = [df_PETPEN_ST_srad, df_PETDYN_ST_srad, df_PETPNO_ST_srad]
Table = table(frames)
Table.style.highlight_max(color='lightgreen', axis=1)

In [ ]:
frames1 =[df_PETASCE_ST_srad]
Table = table(frames1)
Table.style.highlight_max(color='lightgreen', axis=1)

In [ ]:
frames1 =[df_PETPT_ST_srad]
Table = table(frames1)
Table.style.highlight_max(color='lightgreen', axis=1)

## S1

In [ ]:
frames = [df_PETPEN_S1_srad, df_PETDYN_S1_srad, df_PETPNO_S1_srad]
Table = table(frames)
Table.style.highlight_max(color='lightgreen', axis=1)

In [ ]:
frames1 =[df_PETASCE_S1_srad]
Table = table(frames1)
Table.style.highlight_max(color='lightgreen', axis=1)

In [ ]:
frames1 =[df_PETPT_S1_srad]
Table = table(frames1)
Table.style.highlight_max(color='lightgreen', axis=1)

## Final Result : Use "Unrealistic" Parameter Domains

- To get a complete picture of the most dominant parameters in these models, an unrealistic set of parameters is used in each model to determine the overall importance.
- As already mentioned, TMAX and SRAD have the largest Sobol indices in Priestley-Taylor and Penman-Monteith models, respectively. 

In [ ]:
model = 'PETPT'
bounds = {
        'tmax':[-30.0, 60.0],
        'tmin':[-30.0, 60.0],
        'srad': [0.0, 30.0],
        'msalb': [0.0, 1.0],
        'xhlai': [0.0, 20.0]
    }


SM = SensitivityModel(model, bounds, sample_list, method)
decimal=4
df_PETPT_S1_new, df_PETPT_ST_new = SM.generate_dataframe(decimal)

model = 'PETPNO'
bounds = {
    'tmax':[-30.0, 60.0],
    'tmin':[-30.0, 60.0],
    'srad': [0.0, 30.0],
    'msalb': [0.0, 1.0],
    'xhlai': [0.0, 20.0],
    'tavg': [-30, 60],
    'tdew': [-30, 60],
    'windsp': [0.0, 10.0],
    'clouds': [0.0, 1.0]
}

SM = SensitivityModel(model, bounds, sample_list, method)
df_PETPNO_S1_new, df_PETPNO_ST_new = SM.generate_dataframe(decimal)

model = 'PETPEN'
bounds = {
    'tmax':[-30.0, 60.0],
    'tmin':[-30.0, 60.0],
    'srad': [0.0, 30.0],
    'msalb': [0.0, 1.0],
    'xhlai': [0.0, 20.0],
    'tavg': [-30, 60],
    'tdew': [-30, 60],
    'windsp': [1.0, 10.0],
    'windht': [1.0, 25.0],
    'vapr': [0.0, 20.0],
    'clouds': [0.0, 1.0],
    'eoratio': [0.0, 2.0]
}

SM = SensitivityModel(model, bounds, sample_list, method)
df_PETPEN_S1_new, df_PETPEN_ST_new = SM.generate_dataframe(decimal)


model = 'PETDYN'
bounds = {
    'tmax':[-30.0, 60.0],
    'tmin':[-30.0, 60.0],
    'srad': [0.0, 30.0],
    'msalb': [0.0, 1.0],
    'xhlai': [0.0, 20.0],
    'tavg': [-30, 60],
    'tdew': [-30, 60],
    'windsp': [0.0, 10.0],
    'canht': [0.0, 5.0],
    'clouds': [0.0, 1.0]
}

SM = SensitivityModel(model, bounds, sample_list, method)
df_PETDYN_S1_new, df_PETDYN_ST_new = SM.generate_dataframe(decimal)

model = 'PETASCE'
bounds = {
    'tmax':[-30.0, 60.0],
    'tmin':[-30.0, 60.0],
    'srad': [0.0, 30.0],
    'msalb': [0.0, 1.0],
    'xhlai': [0.0, 20.0],
    'tdew': [-30, 60],
    'canht': [0.0, 5.0],
    'windht': [1.0, 25.0],
    'windrun': [0, 900],
    'xlat': [-65.0, 65.0],
    'xelev': [0, 6000],
    'doy': [1, 365],
    'meevp': [0, 1]    
}

SM = SensitivityModel(model, bounds, sample_list, method)
df_PETASCE_S1_new, df_PETASCE_ST_new = SM.generate_dataframe(decimal)

## ST

In [ ]:
frames = [df_PETPT_ST_new, df_PETPNO_ST_new, df_PETPEN_ST_new, df_PETDYN_ST_new, df_PETASCE_ST_new]

Table = table(frames)
Table.style.highlight_max(color='lightgreen', axis=1)

## S1

In [ ]:
frames = [df_PETPT_S1_new, df_PETPNO_S1_new, df_PETPEN_S1_new, df_PETDYN_S1_new, df_PETASCE_S1_new]

Table = table(frames)
Table.style.highlight_max(color='lightgreen', axis=1)